# Pretty Painting with CycleGAN

My introduction to GANs. Trying to recreate the code for image-to-image translation using CycleGAN for monet painting. I'm going to based off from this [Machine Learning Mastery Tutorial](https://machinelearningmastery.com/cyclegan-tutorial-with-keras/).

In [2]:
import tensorflow as tf
import os, json, shutil
from PIL import Image
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import numpy as np
from tensorflow.keras import Model, losses, optimizers
import glob
from tqdm import tqdm
import random

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
ls "data"

 Volume in drive F is Files Volume
 Volume Serial Number is 301F-B35F

 Directory of f:\git\Pretty-Paintings-With-CycleGan\data

12/30/2021  10:57 PM    <DIR>          .
12/30/2021  10:57 PM    <DIR>          ..
10/17/2020  01:31 PM           535,331 metadata.csv
12/30/2021  10:57 PM    <DIR>          testA
12/30/2021  10:57 PM    <DIR>          testB
12/30/2021  10:57 PM    <DIR>          trainA
12/30/2021  10:57 PM    <DIR>          trainB
               1 File(s)        535,331 bytes
               6 Dir(s)  128,586,522,624 bytes free


## Loading in the data

In [5]:
DataDir = "F:\git\Pretty-Paintings-With-CycleGan\data"

train_monet = glob.glob("data/trainA/*")
train_photo = glob.glob("data/trainB/*")
test_monet = glob.glob("data/testA/*")
test_photo = glob.glob("data/testB/*")

print(f"train monet length: {len(train_monet)}")
print(f"train photo length: {len(train_photo)}")
print(f"test monet length: {len(test_monet)}")
print(f"test photo length: {len(test_photo)}")

#variables to store the data
#we're going to say 0 is monet, 1 is a photo
train_x = []
train_y = []
test_x = []
test_y = []

#loading in the images
#for better pratice, should include try catch statement in case the data isn't clean

#I'm going to combine the train and test datasets because it provides more data and 
#the CycleGAN architecture doesn't generate loss based on a validation set, rather it uses another GAN to generate loss function
for img in tqdm(train_monet, desc="loading train Monet"):
    img_array = np.array(Image.open(img))
    train_x.append(img_array)
    #is actual money so we're going to add a 0
    train_y.append(0)

for img in tqdm(train_photo, desc="loading train photo"):
    img_array = np.array(Image.open(img))
    train_x.append(img_array)
    #is actual money so we're going to add a 0
    train_y.append(1)

for img in tqdm(test_monet, desc="loading test monet"):
    img_array = np.array(Image.open(img))
    test_x.append(img_array)
    #is actual money so we're going to add a 0
    test_y.append(0)

for img in tqdm(test_photo, desc="loading test photo"):
    img_array = np.array(Image.open(img))
    test_x.append(img_array)
    #is actual money so we're going to add a 0
    test_y.append(1)

#converting them to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

train monet length: 1072
train photo length: 6287
test monet length: 121
test photo length: 751


loading test photo: 100%|██████████| 751/751 [00:04<00:00, 151.93it/s]


In [6]:
#verifying that data has loaded in correctly
index = random.randint(0, len(train_x))
plt.imshow(train_x[index])
print(train_y[index])
print(f"train input shape: {train_x.shape}")
print(f"train output shape: {train_x.shape}")
print(f"test input shape: {test_x.shape}")
print(f"test output shape: {test_y.shape}")

SyntaxError: f-string: unmatched ')' (Temp/ipykernel_18720/1183165867.py, line 8)

## Image Preprocessing

In [ ]:
#manipulation methods to prevent overfitting

BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

def random_crop(image):
  cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image

# normalizing the images to [-1, 1]
def normalize(image):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  return image

def random_jitter(image):
  # resizing to 286 x 286 x 3
  image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  # randomly cropping to 256 x 256 x 3
  image = random_crop(image)

  # random mirroring
  image = tf.image.random_flip_left_right(image)

  return image